In [37]:

import google.generativeai as genai
from PIL import Image
from dotenv import load_dotenv
import json
import os
from io import BytesIO
import pprint

In [38]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [39]:
# Gemini prompt
GEMINI_PROMPT = """
You are an expert in biometric image validation and body measurement estimation.

Your task is to analyze the input image and return a JSON response with the following keys:

{
  "valid": true/false,
  "full_body": true/false,
  "one_person": true/false,
  "standing_pose": true/false,
  "background_ok": true/false,
  "view": "front" | "back" | "unknown",
  "reasons": {
    "full_body": "...",
    "one_person": "...",
    "standing_pose": "...",
    "background_ok": "...",
    "view": "..."
  },
  "measurements": {
    "height_cm": number | null,
    "chest_cm": number | null,
    "waist_cm": number | null,
    "hip_cm": number | null,
    "inseam_cm": number | null,
  },
    "overall_accuracy_percent": number | null
}

Validation Rules:
- "valid" is true ONLY if all validation checks are true.
- "full_body": Entire body must be visible from head to toe.
- "one_person": Only one person should be visible.
- "standing_pose": Subject must be standing upright.
- "background_ok": Plain or uncluttered background.
- "view": Detect as 'front' (facial features visible), 'back' (no face), or 'unknown'.

Measurement Estimation:
- Estimate measurements using visible body proportions.
- Use known anthropometric ratios if needed.
- If estimation is not possible, set values to null.
- Provide a single "overall_accuracy_percent" between 0-100 indicating confidence in all measurements combined.

Important: Return ONLY valid JSON. Do not include markdown or explanation outside the JSON.
"""


In [40]:
def send_to_gemini_validation(image_path: str) -> dict:
    try:
        image = Image.open(image_path)

        model = genai.GenerativeModel(model_name="gemini-2.0-flash")

        response = model.generate_content(
            [GEMINI_PROMPT.strip(), image],
            generation_config={"temperature": 0.4}
        )

        text = response.text.strip()
        json_start = text.find('{')
        json_end = text.rfind('}') + 1
        json_str = text[json_start:json_end]

        return json.loads(json_str)

    except Exception as e:
        return {"valid": False, "reason": f"Gemini validation failed: {str(e)}"}

In [42]:
if __name__ == "__main__":
    image_path = "../images/front-image.png" 
    result = send_to_gemini_validation(image_path)
    pprint.pprint(result)

{'background_ok': True,
 'full_body': True,
 'measurements': {'chest_cm': 95,
                  'height_cm': 180,
                  'hip_cm': 92,
                  'inseam_cm': 82,
                  'waist_cm': 80},
 'one_person': True,
 'overall_accuracy_percent': 75,
 'reasons': {'background_ok': 'The background is plain and uncluttered.',
             'full_body': 'The entire body is visible from head to toe.',
             'one_person': 'Only one person is present in the image.',
             'standing_pose': 'The person is standing upright.',
             'view': 'Facial features are visible, indicating a front view.'},
 'standing_pose': True,
 'valid': True,
 'view': 'front'}
